In [24]:
from ultralytics import YOLO
import torch
import cv2

In [25]:
torch.cuda.is_available()

True

In [ ]:
#model = YOLO("yolov8l.pt")
model = YOLO("runs/detect/train2/weights/last.pt")

model.train(data="config.yaml", epochs=50, batch = 16, imgsz=480, patience=0)
model.val()

In [26]:
best = YOLO("runs/detect/train2/weights/best.pt")
best.val()

Ultralytics YOLOv8.0.105  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients
val: Scanning C:\Users\dell\Desktop\YOLOV8\data\val\labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.66s/it]
                   all         17         20      0.995          1      0.995      0.866
Speed: 0.0ms preprocess, 24.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\val


ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x000002B317B0DA50>
fitness: 0.8788743421052632
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.86597])
names: {0: 'iced_coffee'}
plot: True
results_dict: {'metrics/precision(B)': 0.9950253406513428, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.995, 'metrics/mAP50-95(B)': 0.8659714912280702, 'fitness': 0.8788743421052632}
save_dir: WindowsPath('runs/detect/val')
speed: {'preprocess': 0.0, 'inference': 24.14042809430291, 'loss': 0.0, 'postprocess': 0.9200292475083295}

In [ ]:
# Model on img
img = cv2.imread("data/coffee.png")

results = best.predict(img, stream=True)                       # run prediction on img
for result in results:                                         # iterate results
    boxes = result.boxes.cpu().numpy()                         # get boxes on cpu in numpy
    for box in boxes:                                          # iterate boxes
        r = box.xyxy[0].astype(int)                            # get corner points as int
        print(r)                                               # print boxes
        cv2.rectangle(img, r[:2], r[2:], (0, 255, 0), 2)       # draw boxes on img
        cls = result.names[int(box.cls[0])]
        cv2.putText(img, cls,(r[0] - 10, r[1] - 10), 0, 0.7, (0,255,0), 2)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
!pip install labelImg
!labelImg

In [ ]:
# Model on Video
green = (0, 255, 0)

# Load the YOLO model
model = YOLO("runs/detect/train2/weights/best.pt")

# Define the class names
class_names = ['iced coffee']

# Set the path to the input video
video_path = "data/Recipe.mp4"
cap = cv2.VideoCapture(video_path)

# Get the frame dimensions and FPS of the input video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Set the output path for the annotated video
output_path = "data/output_video_2.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while True:
    # Read the next frame from the input video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection on the frame using the YOLO model
    results = model.predict(frame, conf=0.25, iou=0.1)

    # Iterate over the detected objects in the frame
    for result in results:
        boxes = result.boxes.cpu().numpy()

        if len(boxes) > 0:
            xSum = 0
            ySum = 0
            for box in boxes:
                # Get the bounding box coordinates and confidence of the detected object
                r = box.xyxy[0].astype(int)
                conf = box.conf[0]

                # Calculate the center coordinates of the bounding box
                center_x = int((r[0] + r[2]) / 2)
                center_y = int((r[1] + r[3]) / 2)

                xSum += center_x
                ySum += center_y

                # Draw the bounding box and center point on the frame
                cv2.rectangle(frame, r[:2], r[2:], green, 2)
                cv2.circle(frame, (center_x, center_y), 5, green, -1)

                # Get the class label of the detected object
                cls = result.names[int(box.cls[0])]
                label = f"{cls}_{conf:.2f}"

                # Draw the class label on the frame
                cv2.putText(frame, label, (r[0] - 10, r[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


            avg_center_x = int(xSum / len(boxes))
            avg_center_y = int(ySum / len(boxes))
            cv2.circle(frame, (avg_center_x, avg_center_y), 5 , (255,0,0), -1)

            # Add the middle and mean circles
            cv2.circle(frame, (frame_width//2,frame_height//2), 5,(255,255,255),-1)
            cv2.line(frame, (avg_center_x, avg_center_y), (frame_width // 2, frame_height // 2), (0, 0, 255), 2)

            # Calculate the x and y differences
            # -x solda demek +y aşağıda demek
            diff_x = avg_center_x - frame_width // 2
            diff_y = avg_center_y - frame_height // 2
            cv2.putText(frame, f"{diff_x} {diff_y}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    # Write the annotated frame to the output video
    output_video.write(frame)

    # Display the annotated frame
    cv2.imshow('Webcam', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture and output video objects
cap.release()
output_video.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


In [ ]:
# Release the video capture and output video objects
cap.release()
output_video.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

In [ ]:
# overlap olma durumuna bak